_This is devoted to detecting non-human users in our database_

From http://www.erinshellman.com/bot-or-not/

Follower distributions

    Fast-forward to clean, well-formatted data and it doesn’t take long to find fishiness. On average, bots follow 1400 people whereas humans follow 500. Bots are similarly strange in their distribution of followers. Humans have a fairly uniform distribution of followers. Some people are popular, some not so much, and many in between. Conversely, these bots are extremely unpopular with an average of a measly 28 followers.
    
Lexical diversity

    Again these bots look strange. Humans have a beautiful, almost textbook normal distribution of diversities centered at 0.70. Bots on the other hand have more mass at the extremes, especially towards one. A lexical diversity of one means that every word in the document is unique, implying that bots are either not tweeting much, or are tweeting random strings of text.

In [1]:
import pandas as pd

#Plotting 
%matplotlib inline
from matplotlib import pyplot as plt
import seaborn as sns
sns.set(style="whitegrid")


%cd twitterproject

# inject config value (on command line would've been --config=data-analysis)
import sys
# args = ['--config', 'data-analysis']
args = ['--config', 'laptop-mining']
old_sys_argv = sys.argv
sys.argv = [old_sys_argv[0]] + args
import environment

/Users/adam/.pyenv/versions/3.6.4/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/adam/.pyenv/versions/3.6.4/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


(bookmark:twitterproject) -> /Users/adam/Dropbox/PainNarrativesLab/TwitterProject
/Users/adam/Dropbox/PainNarrativesLab/TwitterProject
['/Users/adam/.pyenv/versions/3.6.4/lib/python3.6/site-packages/ipykernel_launcher.py', '--config', 'laptop-mining']
Reading configuration from /Users/adam/Dropbox/PainNarrativesLab/TwitterProject/configurations/laptop-mining.config.ini


In [ ]:
from TwitterDatabase.Repositories import DataRepositories as DR
from TwitterDatabase.DatabaseAccessObjects import DataConnections as DC
from TwitterDatabase.Models.WordORM import Word
from TwitterDatabase.Models.TweetORM import Users as User
from TwitterDatabase.Models.TweetORM import Tweet
from DataAnalysis.SearchTools.WordMaps import get_adjacent_word_counts, get_adjacent_words, get_user_ids_for_word

EXP_TERMS_FILEPATH = '%s/experimental-terms.xlsx' % environment.EXPERIMENTS_FOLDER
IDS_FILEPATH = "%s/temp_output/user-ids.xlsx" % environment.LOG_FOLDER_PATH

# Bot detection on users

## Load data

In [ ]:
dao = DC.MySqlConnection(environment.CREDENTIAL_FILE)

In [ ]:
data= pd.read_sql_query("select userID, friends_count, followers_count from users", dao.engine, index_col='userID')
# Because someone set this field to string
data.followers_count = data.apply(lambda x : int(x.followers_count), axis=1)

In [ ]:
data.describe()

## Prune the data

In [ ]:
MIN_FRIENDS = 1
MIN_FOLLOWERS =0

# cutoff the top 1%
MAX_PERCENTILE = 0.99

start_count = len(data)

# Trim by number of friends
friends_cutoff = data.friends_count.quantile(MAX_PERCENTILE)
data = data[data.friends_count.between(MIN_FRIENDS, friends_cutoff, inclusive=True)]
# Trim by number of followers
followers_cutoff = data.followers_count.quantile(MAX_PERCENTILE)
data = data[data.followers_count.between(MIN_FOLLOWERS, followers_cutoff, inclusive=True)]

print("Cutoff for friend count: %s \nCutoff for follower count: %s \nRemoved %s users" % (friends_cutoff, followers_cutoff, start_count - len(data)))

### Number of people the user follows (friends)

In [ ]:
sns.distplot(data['friends_count'])

In [ ]:
sns.violinplot(data.friends_count)

### How many people follow the user

In [ ]:
sns.distplot(data.followers_count)

In [ ]:
sns.violinplot(data.followers_count)

In [ ]:
data.sort_values('followers_count', ascending=True)[:10]

## Distribution of status updates


In [ ]:

query = "SELECT userID, screen_name, statuses_count FROM users ORDER BY statuses_count DESC"
freq = pd.read_sql_query(query, dao.engine, index_col='userID')

In [ ]:
sns.distplot(freq.statuses_count)

In [ ]:
freq.statuses_count.describe()

In [ ]:
for i in range(80, 100):
    j = round(i * 0.01,2)
    q = freq.statuses_count.quantile(j)
    print("%s th : %s" % (i, int(q))) 

## Calculate relationships

In [ ]:
# Subtract followers from friends
data['friends_less_followers'] = data.apply(lambda x : x.friends_count - x.followers_count, axis=1)

def f(row):
    if row.followers_count == 0: return 0
    return row.friends_count / row.followers_count
data['ratio_friends_followers'] = data.apply(lambda x: f(x), axis=1)

In [ ]:
data.describe()

In [ ]:
data[:10]

### Difference between friends and followers

In [ ]:
data.sort_values('friends_less_followers', ascending=True)[:10]

In [ ]:
sns.violinplot(data.friends_less_followers)

In [ ]:
sns.distplot(data.friends_less_followers)

### Ratio of friends to followers

In [ ]:
sns.violinplot(data.ratio_friends_followers)

In [ ]:
data.sort_values('ratio_friends_followers', ascending=False)[:10]

# Bot detection at tweet level

Finding bots based only on the data in tweets

## New mining (where tweet stores user data)